In [2]:
import pandas as pd
import numpy as np
import cv2

In [3]:
import tensorflow as tf
from keras import layers, models
from keras.preprocessing.image import ImageDataGenerator

# Creating Generators

In [25]:
# Define image dimensions
img_width, img_height = 224, 224

# Define number of classes
num_classes = 8

# Define batch size
batch_size = 32

# Define number of epochs
epochs = 10

# Define paths to training and validation data
train_data_dir = 'data/train.csv'
test_data_dir = 'data/test.csv'

train_df = pd.read_csv(train_data_dir)
test_df = pd.read_csv(test_data_dir)

# cast the labels to categorical
train_df = train_df.astype({'label': 'string'})
test_df = test_df.astype({'label': 'string'})

# Data preprocessing and augmentation
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input,
    validation_split=0.2
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input        
)

train_images = train_generator.flow_from_dataframe(train_df,
                                                    x_col='path_img',
                                                    y_col='label',
                                                    target_size=(img_width, img_height),
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                    batch_size=batch_size,
                                                    shuffle = True,
                                                    seed = 42,
                                                    subset = 'training')

val_images = train_generator.flow_from_dataframe(train_df,
                                                        x_col='path_img',
                                                        y_col='label',
                                                        target_size=(img_width, img_height),
                                                        color_mode='rgb',
                                                        class_mode='categorical', 
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        seed=42,
                                                        subset='validation')

test_generator = test_generator.flow_from_dataframe(test_df,
                                                        x_col='path_img',
                                                        y_col='label',
                                                        target_size=(img_width, img_height),
                                                        color_mode='rgb',
                                                        class_mode='categorical',
                                                        batch_size=batch_size,
                                                        shuffle=False)

Found 8960 validated image filenames belonging to 8 classes.
Found 2240 validated image filenames belonging to 8 classes.
Found 4800 validated image filenames belonging to 8 classes.


# MODELING

In [26]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(img_width, img_height, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

In [29]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(8, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

# TRAINING

In [30]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data= val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
129/280 [============>.................] - ETA: 2:29 - loss: 0.7583 - accuracy: 0.7490

c:\Users\crist\universidad\3_PI_CNN\ProyIng_Grupo2\env\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


280/280 [==============================] - 341s 1s/step - loss: 0.6373 - accuracy: 0.7934 - val_loss: 0.5039 - val_accuracy: 0.8330
Epoch 2/100
280/280 [==============================] - ETA: 0s - loss: 0.3857 - accuracy: 0.8752

KeyboardInterrupt: 

# RESULTS


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
results = model.evaluate(test_images, verbose=0)
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
predictions = np.argmax(model.predict(test_images), axis=1)

cm = confusion_matrix(test_images.labels, predictions)
clr = classification_report(test_images.labels, predictions, target_names=test_images.class_indices, zero_division=0)

In [ ]:
plt.figure(figsize=(30, 30))
sns.heatmap(cm, annot=True, fmt='g', vmin=0, cmap='Blues', cbar=False)
plt.xticks(ticks=np.arange(101) + 0.5, labels=test_images.class_indices, rotation=90)
plt.yticks(ticks=np.arange(101) + 0.5, labels=test_images.class_indices, rotation=0)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
print("Classification Report:\n----------------------\n", clr)
